# Street Fighter Tutorial
This notebook accompanies the YouTube tutorial on <a href='https://www.youtube.com/c/NicholasRenotte'>Nicholas Renotte</a>

# Setup StreetFighter

In [6]:
%pip install gym==0.25.2
%pip install --upgrade setuptools==66
%pip install importlib-metadata==4.8.1
%pip install pyglet==1.3.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gym
Failed to build gym


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [481 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib
      creating build\lib\gym
      copying gym\core.py -> build\lib\gym
      copying gym\error.py -> build\lib\gym
      copying gym\logger.py -> build\lib\gym
      copying gym\version.py -> build\lib\gym
      copying gym\__init__.py -> build\lib\gym
      creating build\lib\gym\envs
      copying gym\envs\registration.py -> build\lib\gym\envs
      copying gym\envs\__init__.py -> build\lib\gym\envs
      creating build\lib\gym\spaces
      copying gym\spaces\box.py -> build\lib\gym\spaces
      copying gym\spaces\dict.py -> build\lib\gym\spaces
      copying gym\spaces\discrete.py -> build\lib\gym\spaces
      copying gym\spaces\multi_binary.py -> build\lib\gym\spaces
      copying gym\spaces\multi_discrete.py -> build\lib\gym\

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Import retro to play Street Fighter using a ROM
import retro
# Import time to slow down game
import time

In [8]:
# See the different retro games
retro.data.list_games()

['1942-Nes',
 '1943-Nes',
 '3NinjasKickBack-Genesis',
 '8Eyes-Nes',
 'AaahhRealMonsters-Genesis',
 'AbadoxTheDeadlyInnerWar-Nes',
 'AcceleBrid-Snes',
 'ActRaiser2-Snes',
 'ActionPachio-Snes',
 'AddamsFamily-GameBoy',
 'AddamsFamily-Genesis',
 'AddamsFamily-Nes',
 'AddamsFamily-Sms',
 'AddamsFamily-Snes',
 'AddamsFamilyPugsleysScavengerHunt-Nes',
 'AddamsFamilyPugsleysScavengerHunt-Snes',
 'AdvancedBusterhawkGleylancer-Genesis',
 'Adventure-Atari2600',
 'AdventureIsland-GameBoy',
 'AdventureIsland3-Nes',
 'AdventureIslandII-Nes',
 'AdventuresOfBatmanAndRobin-Genesis',
 'AdventuresOfBayouBilly-Nes',
 'AdventuresOfDinoRiki-Nes',
 'AdventuresOfDrFranken-Snes',
 'AdventuresOfKidKleets-Snes',
 'AdventuresOfMightyMax-Genesis',
 'AdventuresOfMightyMax-Snes',
 'AdventuresOfRockyAndBullwinkleAndFriends-Genesis',
 'AdventuresOfRockyAndBullwinkleAndFriends-Nes',
 'AdventuresOfRockyAndBullwinkleAndFriends-Snes',
 'AdventuresOfStarSaver-GameBoy',
 'AdventuresOfYogiBear-Snes',
 'AeroFighters-Snes',
 

In [9]:
# python -m retro.import . # Run this from the roms folder, or where you have your game roms 

In [10]:
# Starts up the game environment
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

RuntimeError: Cannot create multiple emulator instances per process, make sure to call env.close() on each environment before creating a new one

In [ ]:
# Closes the game environment - important given we can only run one at a time 
# env.close()

In [ ]:
# Sample the observation space
env.observation_space

Box(0, 255, (200, 256, 3), uint8)

In [ ]:
# Sample the actions that are avaialble - MultiBinary
env.action_space.sample()

array([0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        print(reward)

ModuleNotFoundError: No module named 'gym.envs.classic_control.rendering'

In [ ]:
env.close()

In [ ]:
info

# Setup Environment
## What we are going to do! FUNNN
- Observation Preprocess - grayscale (DONE), frame delta, resize the frame so we have less pixels  (DONE) 
- Filter the action - parameter DONE
- Reward function - set this to the score

In [ ]:
%pip install opencv-python

In [ ]:
# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
# Import matplotlib for plotting the image
from matplotlib import pyplot as plt

In [ ]:
# Create custom environment 
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [ ]:
env = StreetFighter()

RuntimeError: Cannot create multiple emulator instances per process, make sure to call env.close() on each environment before creating a new one

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space.shape

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward > 0: 
            print(reward)

In [ ]:
obs = env.reset()

In [ ]:
obs, reward, done, info = env.step(env.action_space.sample())

In [ ]:
plt.imshow(cv2.cvtColor(obs, cv2.COLOR_BGR2RGB))

# Hyperparameter tune

In [ ]:
%pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install optuna

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rich 6.2.0 requires typing-extensions<4.0.0,>=3.7.4, but you have typing-extensions 4.7.1 which is incompatible.



  Using cached optuna-3.4.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.12.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached SQLAlchemy-2.0.23-cp37-cp37m-win_amd64.whl.metadata (9.8 kB)
  Using cached PyYAML-6.0.1-cp37-cp37m-win_amd64.whl.metadata (2.1 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached greenlet-3.0.1-cp37-cp37m-win_amd64.whl.metadata (3.8 kB)
  Using cached MarkupSafe-2.1.3-cp37-cp37m-win_amd64.whl.metadata (3.1 kB)
Using cached optuna-3.4.0-py3-none-any.whl (409 kB)
Using cached alembic-1.12.1-py3-none-any.whl (226 kB)
Using cached SQLAlchemy-2.0.23-cp37-cp37m-win_amd64.whl (2.1 MB)
Using cached PyYAML-6.0.1-cp37-cp37m-win_amd64.whl (153 kB)
Using cached greenlet-3.0.1-cp37-cp37m-win_amd64.whl (287 kB)
Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
Using cached MarkupSafe-2.1.3-cp37-cp3

In [ ]:
%pip install stable-baselines3[extra]

  Using cached stable_baselines3-2.1.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached rich-13.6.0-py3-none-any.whl.metadata (18 kB)
  Using cached Shimmy-1.1.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
  Using cached AutoROM.accept_rom_license-0.6.1-py3-none-any.whl
  Using cached ale_py-0.8.1-cp38-cp38-win_amd64.whl (952 kB)
  Using cached google_auth-2.23.4-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.5.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Using cached Shimmy-1.1.0-py3-none-any.whl (37 kB)
Using cached tensorboard-2.14.0-py3-none-any.whl (5.5 MB)
Using cached rich-13.6.0-py3-none-any.whl (239 kB)
Using cached stable_baselines3-2.1.0-py3-non

In [1]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

c:\Users\vk\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [3]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [4]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [5]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=30000)
        #model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [6]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')
# study.optimize(optimize_agent, n_trials=10, n_jobs=1)
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2023-11-14 00:50:43,048] A new study created in memory with name: no-name-8c99e2f4-9e6c-40b3-968a-f99faa77c4c1
C:\Users\vk\AppData\Local\Temp\ipykernel_9400\3802630500.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
C:\Users\vk\AppData\Local\Temp\ipykernel_9400\3802630500.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
C:\Users\vk\AppData\Local\Temp\ipykernel_9400\3802630500.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0

In [ ]:
study.best_params

{'n_steps': 6177,
 'gamma': 0.9141035935776617,
 'learning_rate': 4.555647896453844e-05,
 'clip_range': 0.17950577872799717,
 'gae_lambda': 0.8532103042940042}

In [ ]:
study.best_trial

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[-1000.0], datetime_start=datetime.datetime(2023, 11, 13, 22, 45, 46, 417448), datetime_complete=datetime.datetime(2023, 11, 13, 22, 45, 46, 454464), params={'n_steps': 6177, 'gamma': 0.9141035935776617, 'learning_rate': 4.555647896453844e-05, 'clip_range': 0.17950577872799717, 'gae_lambda': 0.8532103042940042}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=1), 'gamma': FloatDistribution(high=0.9999, log=True, low=0.8, step=None), 'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, trial_id=0, value=None)

In [ ]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_5_best_model.zip'))

FileNotFoundError: [Errno 2] No such file or directory: 'opt\\trial_5_best_model.zip.zip'

# Setup Callback

In [ ]:
# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train Model

In [ ]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

In [ ]:
# Reload previous weights from HPO
model.load(os.path.join(OPT_DIR, 'trial_5_best_model.zip'))

In [ ]:
# Kick off training 
model.learn(total_timesteps=100000, callback=callback)
# model.learn(total_timestep=5000000) 

# Evaluate the Model

In [ ]:
model = PPO.load('./train/best_model_10000.zip')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

In [ ]:
mean_reward

# Test out the Model

In [ ]:
obs = env.reset()

In [ ]:
obs.shape

In [ ]:
env.step(model.predict(obs)[0])

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)